# MArch MAdness Models, 2024

Data From Kaggle Competition Files

Year two of trying this. Going to use the actual evaluation Functions intead of just beating my frined's picks

### PAckages

In [2]:
import os
import pandas as pd
import numpy as np
import sklearn
import random
import time

In [3]:
os.chdir('C:\\Users\\Nick\\Python Projects\\2024 March Madness\\MMML')

### Data Import

In [7]:
# I want Mteams, MRegularSeasonDetailedResults, MTeamConferences, MTourneyCompactResults, MTeamConferences
# MMasseyOrdinals, MTeamSpellings, 2024_tourney_seeds (not updated id bet)

MTeams = pd.read_csv('Data\\MTeams.csv')
WTeams = pd.read_csv('Data\\WTeams.csv')

MRegSeason = pd.read_csv('Data\\MRegularSeasonDetailedResults.csv')
WRegSeason = pd.read_csv('Data\\WRegularSeasonDetailedResults.csv')

MConf = pd.read_csv('Data\\MTeamConferences.csv')
WConf = pd.read_csv('Data\\WTeamConferences.csv')

MTourneyResults = pd.read_csv('Data\\MNCAATourneyCompactResults.csv')
WTourneyResults = pd.read_csv('Data\\WNCAATourneyCompactResults.csv')

#MMassey = pd.read_csv('Data\\MMasseyOrdinals.csv')

MConfTOurney = pd.read_csv('Data\\MConferenceTourneyGames.csv')


In [8]:
RegSeason = pd.concat([MRegSeason, WRegSeason], ignore_index = True)
RegSeason.head(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [12]:

SeasonWins = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].count().reset_index().rename(columns = {'LTeamID':'Wins', 'WTeamID':'TeamID'})
SeasonLs = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].count().reset_index().rename(columns = {'WTeamID':'Losses', 'LTeamID':'TeamID'})
OpponentsW = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].apply(list).reset_index().rename(columns = {'LTeamID':'OpponentsBeaten', 'WTeamID':'TeamID'})
OpponentsL = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].apply(list).reset_index().rename(columns = {'WTeamID':'OpponentsLost', 'LTeamID':'TeamID'})


ConfSeasonWins = MConfTOurney.groupby(['WTeamID', 'Season'])['LTeamID'].count().reset_index().rename(columns = {'LTeamID':'ConfWins', 'WTeamID':'TeamID'})
ConfSeasonLs = MConfTOurney.groupby(['LTeamID', 'Season'])['WTeamID'].count().reset_index().rename(columns = {'WTeamID':'ConfLosses', 'LTeamID':'TeamID'})
ConfOpponentsW = MConfTOurney.groupby(['WTeamID', 'Season'])['LTeamID'].apply(list).reset_index().rename(columns = {'LTeamID':'ConfOpponentsBeaten', 'WTeamID':'TeamID'})
ConfOpponentsL = MConfTOurney.groupby(['LTeamID', 'Season'])['WTeamID'].apply(list).reset_index().rename(columns = {'WTeamID':'ConfOpponentsLost', 'LTeamID':'TeamID'})


#Merge these Columns
merged_df = pd.merge(SeasonWins, SeasonLs, on=['TeamID', 'Season'], how='outer')

merged_df1 = pd.merge(merged_df, OpponentsW, on=['TeamID', 'Season'], how='left')

merged_df2 = pd.merge(merged_df1, OpponentsL, on=['TeamID', 'Season'], how='left')

merged_df3 = pd.merge(merged_df2, ConfSeasonWins, on=['TeamID', 'Season'], how='left')
merged_df4 = pd.merge(merged_df3, ConfOpponentsW, on=['TeamID', 'Season'], how='left')
merged_df5 = pd.merge(merged_df4, ConfOpponentsL, on=['TeamID', 'Season'], how='left')
TeamRecords = pd.merge(merged_df5, ConfSeasonLs, on=['TeamID', 'Season'], how='left')



TeamRecords.head(50)

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",NaN,NaN,NaN,NaN
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",NaN,NaN,NaN,NaN
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",NaN,NaN,NaN,NaN
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",NaN,NaN,NaN,NaN
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",NaN,NaN,NaN,NaN
5,1101,2019,23.0,6.0,"[1117, 1176, 1189, 1334, 1415, 1144, 1369, 130...","[1337, 1403, 1358, 1372, 1358, 1368]",2.0,"[1368, 1309]",NaN,NaN
6,1101,2020,16.0,11.0,"[1369, 1309, 1311, 1249, 1394, 1368, 1146, 123...","[1180, 1337, 1424, 1402, 1374, 1455, 1270, 135...",NaN,NaN,NaN,NaN
7,1101,2021,19.0,4.0,"[1190, 1122, 1303, 1470, 1249, 1223, 1368, 114...","[1403, 1116, 1358, 1146]",2.0,"[1249, 1311]",NaN,NaN
8,1101,2022,19.0,10.0,"[1426, 1230, 1180, 1167, 1255, 1430, 1469, 146...","[1428, 1401, 1372, 1358, 1213, 1308, 1370, 135...",3.0,"[1430, 1372, 1370]",[1308],1.0
9,1101,2023,9.0,17.0,"[1238, 1319, 1477, 1167, 1470, 1426, 1469, 137...","[1401, 1460, 1451, 1415, 1243, 1372, 1381, 141...",NaN,NaN,[1465],1.0


Deal With the winless and undefeated seasons

In [18]:
TeamRecords['Wins'].fillna(0, inplace = True)
TeamRecords['Losses'].fillna(0, inplace = True)
TeamRecords['ConfWins'].fillna(0, inplace = True)
TeamRecords['ConfLosses'].fillna(0, inplace = True)

In [19]:
TeamRecords['Win_PCT'] = (TeamRecords['Wins']+TeamRecords['ConfWins'])/(TeamRecords['Wins'] + TeamRecords['Losses']+TeamRecords['ConfWins'] + TeamRecords['ConfLosses'])
#TeamRecords

In [20]:
# Define a function to fill NaN values with an empty list
def fill_na_with_empty_list(value):
    if isinstance(value, float) and np.isnan(value):
        return []
    else:
        return value

# Fill NaN values with an empty list using applymap() and the defined function
TeamRecords = TeamRecords.applymap(fill_na_with_empty_list)

In [21]:
TeamRecords.isna().sum()

TeamID                 0
Season                 0
Wins                   0
Losses                 0
OpponentsBeaten        0
OpponentsLost          0
ConfWins               0
ConfOpponentsBeaten    0
ConfOpponentsLost      0
ConfLosses             0
Win_PCT                0
dtype: int64

In [22]:
TeamRecords

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,Win_PCT
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,0.095238
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,0.250000
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,0.333333
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,0.360000
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,0.444444
...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,0.000000
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,0.000000
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,0.000000
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,0.000000


Season stats need to be by team so I cant just group by Wteam because then I'd only have their stats from games they won. 
I also can't have Wteam columns in the game and tourney games data when I  train/make predictions. I need to somehow randomize this so sometimes Wteam is T1 and sometimes T2

In [23]:
RegSeason.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [24]:
RegSeasonSwitch = RegSeason[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore',  'WLoc',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
RegSeasonSwitch

,Season,DayNum,LTeamID,LScore,WTeamID,WScore,WLoc,NumOT,LFGM,LFGA,...,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189433,2024,131,3372,74,3465,75,N,0,27,65,...,24,20,25,7,30,13,11,7,4,15
189434,2024,132,3283,75,3179,76,N,0,31,69,...,17,17,17,7,18,16,19,9,3,11
189435,2024,132,3392,60,3180,68,N,0,19,62,...,19,11,14,6,22,17,21,9,4,21
189436,2024,132,3131,55,3221,61,H,0,21,49,...,26,17,25,12,22,10,10,6,3,12


In [25]:
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'H', 'WLoc'] = 'A'
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'A', 'WLoc'] = 'H'
RegSeason.columns.values[6] = 'location'
RegSeasonSwitch.columns.values[6] = 'location'

RegSeason.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(RegSeason.columns)]
RegSeasonSwitch.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(RegSeasonSwitch.columns)]

In [26]:
RegSeason.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [27]:
RegSeasonSwitch.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1236,62,1161,80,H,0,19,41,...,8,32,39,13,18,14,17,11,1,25
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1296,56,1458,84,H,0,23,52,...,17,15,19,14,22,11,6,12,0,13


Looks like theyre still in order. I thought about just combining them and having 2 rows per game, but im thinking thats probably not a great idea since id be training on games that will also be in the test set, so im going to randomly select 50 percent from RegSeason where team 1 is the willer and the remaining indices will come from RegSeasonSwitch where team2 is the winner


In [28]:
RegSeason.shape

(189438, 34)

In [29]:
RegSeasonSwitch.shape

(189438, 34)

In [30]:
half = RegSeason.shape[0]//2

print(half)
np.random.seed(123)
half_indices = random.sample(list(RegSeason.index), k=half)

RegSeasonHalf = RegSeason.iloc[half_indices].sort_values(['Season', 'DayNum'])
#print(RegSeasonHalf.head(20))

RegSeasonSwitchHalf = RegSeasonSwitch.drop(half_indices).sort_values(['Season', 'DayNum'])
#RegSeasonSwitchHalf.head(20)

94719
     Season  DayNum  T1_TeamID  T1_Score  T2_TeamID  T2_Score location  NumOT  \
1      2003      10       1272        70       1393        63        N      0   
4      2003      11       1400        77       1208        71        N      0   
6      2003      12       1161        80       1236        62        H      0   
9      2003      12       1458        84       1296        56        H      0   
10     2003      13       1166       106       1426        50        H      0   
12     2003      13       1237        66       1135        65        N      0   
11     2003      13       1202        74       1106        73        N      0   
24     2003      14       1462        87       1389        48        H      0   
16     2003      14       1161        81       1194        56        H      0   
20     2003      14       1323        89       1237        45        H      0   
27     2003      15       1196        76       1256        55        H      0   
32     2003      17   

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
13,2003,13,1125,48,1323,76,H,0,18,64,...,23,16,23,8,35,18,13,14,19,13
14,2003,14,1135,77,1125,83,N,1,28,60,...,31,12,15,15,18,22,16,18,5,21
15,2003,14,1236,71,1156,78,N,0,23,50,...,18,14,24,8,25,18,15,2,6,18
17,2003,14,1202,57,1166,82,H,0,23,55,...,19,8,14,9,23,21,17,14,5,18


In [31]:
RegSeasonGames = pd.concat([RegSeasonHalf, RegSeasonSwitchHalf]).sort_index()
RegSeasonGames.head(25)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [32]:
RegSeasonGames.shape

(189438, 34)

## COmputing Season Stats By Team

Now to compute season averages. I Want to group by Team 1. I guess i can use a df with every game twice since theyre just averages


In [33]:
RegSeasonDouble = pd.concat([RegSeason, RegSeasonSwitch])
RegSeasonDouble.shape

(378876, 34)

In [34]:
RegSeasonDouble.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF'],
      dtype='object')

In [35]:
RegSeasonDouble.columns = [x.replace('T1_','').replace('T2_','Opp_') for x in list(RegSeasonDouble.columns)]
RegSeasonDouble.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'Opp_TeamID', 'Opp_Score',
       'location', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF'],
      dtype='object')

### Look Into Calculating some more advanced stats

Below is just very basic averages

In [36]:
RegSeasonDouble['Games'] = 1

In [37]:
StatsAvgs = RegSeasonDouble.groupby(['TeamID', 'Season'])[['Score', 'Opp_Score',
       'location','Games', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF']].mean().reset_index()

In [38]:
StatsAvgs.columns

Index(['TeamID', 'Season', 'Score', 'Opp_Score', 'Games', 'NumOT', 'FGM',
       'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM',
       'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF'],
      dtype='object')

## TO DO

Figure out which columns to keep that we want in the season info for each team
Rename columns to specify they are mean values

In [39]:
SA = StatsAvgs[['TeamID', 'Season', 'Score', 'Opp_Score', 'FGM', 'FGM3', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGM3', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF']]

In [40]:
SA.columns = ['TeamID', 'Season', 'P_PG', 'PA_PG', 'FGM_PG', 'FGM3_PG', 'OR_PG', 'DR_PG', 'Ast_PG', 'TO_PG', 'Stl_PG',
       'Blk_PG', 'PF_PG', 'Opp_FGM_PG', 'Opp_FGM3_PG', 'Opp_OR_PG', 'Opp_DR_PG', 'Opp_Ast_PG', 'Opp_TO_PG', 'Opp_Stl_PG',
       'Opp_Blk_PG', 'Opp_PF_PG']

In [41]:
SA

,TeamID,Season,P_PG,PA_PG,FGM_PG,FGM3_PG,OR_PG,DR_PG,Ast_PG,TO_PG,...,PF_PG,Opp_FGM_PG,Opp_FGM3_PG,Opp_OR_PG,Opp_DR_PG,Opp_Ast_PG,Opp_TO_PG,Opp_Stl_PG,Opp_Blk_PG,Opp_PF_PG
0,1101,2014,63.142857,78.619048,20.333333,6.666667,8.000000,20.333333,10.000000,15.000000,...,21.571429,27.142857,6.000000,10.380952,24.095238,15.571429,12.142857,7.000000,5.000000,18.666667
1,1101,2015,61.000000,71.857143,21.428571,7.250000,8.250000,19.642857,11.857143,12.821429,...,20.607143,25.035714,6.178571,10.035714,25.892857,12.928571,13.464286,5.857143,4.250000,16.535714
2,1101,2016,69.851852,76.259259,23.814815,6.851852,8.185185,22.518519,12.888889,13.407407,...,22.370370,26.481481,5.592593,8.592593,25.222222,12.851852,13.444444,5.111111,3.296296,19.666667
3,1101,2017,67.880000,72.640000,24.440000,7.080000,7.560000,22.880000,13.600000,14.480000,...,22.160000,24.840000,7.080000,10.640000,25.040000,13.600000,14.800000,6.040000,3.160000,17.240000
4,1101,2018,70.666667,72.814815,25.518519,6.481481,9.037037,23.222222,13.888889,14.407407,...,21.037037,24.777778,6.629630,9.925926,25.037037,12.037037,15.481481,6.518519,2.925926,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,59.178571,65.428571,21.571429,6.178571,8.678571,22.571429,14.000000,14.535714,...,14.964286,25.178571,5.000000,8.785714,22.964286,11.785714,11.214286,7.892857,3.214286,16.142857
12853,3476,2024,52.200000,68.966667,18.933333,5.500000,9.033333,21.333333,11.533333,18.400000,...,14.433333,26.900000,5.333333,11.866667,24.333333,13.500000,13.466667,10.833333,3.233333,16.033333
12854,3477,2023,64.625000,69.625000,22.937500,3.656250,11.312500,23.875000,11.875000,16.718750,...,15.437500,26.406250,6.531250,9.593750,23.406250,15.437500,15.781250,9.000000,4.062500,18.562500
12855,3477,2024,67.620690,74.172414,24.448276,6.931034,7.620690,25.931034,10.517241,13.000000,...,15.965517,27.655172,5.689655,12.482759,30.896552,13.931034,13.344828,6.724138,3.586207,15.931034


In [42]:
StatsTotals = RegSeasonDouble.groupby(['TeamID', 'Season'])[['Score', 'Opp_Score',
       'location', 'Games', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF']].sum().reset_index()
StatsTotals

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1101,2014,1326,1651,21,3,427,1053,140,375,...,340,385,542,218,506,327,255,147,105,392
1,1101,2015,1708,2012,28,0,600,1482,203,537,...,468,437,636,281,725,362,377,164,119,463
2,1101,2016,1886,2059,27,2,643,1456,185,509,...,409,478,674,232,681,347,363,138,89,531
3,1101,2017,1697,1816,25,4,611,1333,177,477,...,490,397,595,266,626,340,370,151,79,431
4,1101,2018,1908,1966,27,1,689,1525,175,539,...,533,449,633,268,676,325,418,176,79,477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,1657,1832,28,1,604,1563,173,567,...,443,282,405,246,643,330,314,221,90,452
12853,3476,2024,1566,2069,30,0,568,1662,165,597,...,527,295,438,356,730,405,404,325,97,481
12854,3477,2023,2068,2228,32,1,734,1863,117,456,...,633,329,468,307,749,494,505,288,130,594
12855,3477,2024,1961,2151,29,1,709,1877,201,613,...,562,382,545,362,896,404,387,195,104,462


In [43]:
StatsTotals['FG_PCT'] = StatsTotals['FGM']/StatsTotals['FGA']
StatsTotals['FG_PCT3'] = StatsTotals['FGM3']/StatsTotals['FGA3']
StatsTotals['FT_PCT'] = StatsTotals['FTM']/StatsTotals['FTA']

StatsTotals['Opp_FG_PCT'] = StatsTotals['Opp_FGM']/StatsTotals['Opp_FGA']
StatsTotals['Opp_FG_PCT3'] = StatsTotals['Opp_FGM3']/StatsTotals['Opp_FGA3']
StatsTotals['Opp_FT_PCT'] = StatsTotals['Opp_FTM']/StatsTotals['Opp_FTA']

#A possesion can only end with a shot, turnover, foul (free trhrows) i think. We consider an offensive rebound a continuation of that possesion
StatsTotals['POSS'] = StatsTotals['FGA']+StatsTotals['TO']+(StatsTotals['FTA']*0.44)-StatsTotals['OR']
StatsTotals['POSS_PER_GAME'] = StatsTotals['POSS']/StatsTotals['Games']

# THeoretically opponent POSS should only differer by 1 or 2 at most, but since Im calculating it in kind of a janky way, I want to see how much difference there is.
StatsTotals['Opp_POSS'] = StatsTotals['Opp_FGA']+StatsTotals['Opp_TO']+(StatsTotals['Opp_FTA']*0.44)-StatsTotals['Opp_OR']
StatsTotals['Opp_POSS_PER_GAME'] = StatsTotals['Opp_POSS']/StatsTotals['Games']

StatsTotals['POSS_Diff'] = (StatsTotals['POSS'] -StatsTotals['Opp_POSS'])/StatsTotals['Games'] 
 
StatsTotals.sort_values('POSS_Diff').loc[StatsTotals.Season == 2024].tail(25)

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,FG_PCT3,FT_PCT,Opp_FG_PCT,Opp_FG_PCT3,Opp_FT_PCT,POSS,POSS_PER_GAME,Opp_POSS,Opp_POSS_PER_GAME,POSS_Diff
11925,3401,2024,2129,1818,31,0,819,1999,160,573,...,0.279232,0.678279,0.371429,0.294000,0.717602,2245.72,72.442581,2208.48,71.241290,1.201290
8407,3160,2024,2340,1974,31,0,883,1919,210,587,...,0.357751,0.728000,0.402650,0.339823,0.769539,2271.00,73.258065,2233.56,72.050323,1.207742
10262,3286,2024,1938,1915,33,1,684,1814,151,558,...,0.270609,0.756318,0.389205,0.313636,0.704167,2282.76,69.174545,2242.20,67.945455,1.229091
5176,1351,2024,2196,2241,31,3,790,1838,174,517,...,0.336557,0.752981,0.426519,0.344428,0.695455,2171.28,70.041290,2132.40,68.787097,1.254194
710,1138,2024,2019,2388,30,1,743,1786,203,739,...,0.274696,0.711207,0.475216,0.365031,0.724203,2145.16,71.505333,2107.52,70.250667,1.254667
11477,3371,2024,2013,1856,31,3,748,1799,176,553,...,0.318264,0.749451,0.407363,0.321767,0.689498,2133.20,68.812903,2093.72,67.539355,1.273548
8868,3191,2024,2011,2595,32,4,693,1920,155,589,...,0.263158,0.712121,0.469009,0.302932,0.733434,2525.40,78.918750,2484.16,77.630000,1.288750
798,1142,2024,1957,2309,31,3,693,1718,193,584,...,0.330479,0.703911,0.461583,0.355769,0.747222,2114.28,68.202581,2073.80,66.896774,1.305806
12225,3421,2024,1685,1809,29,3,623,1724,115,481,...,0.239085,0.699784,0.394231,0.295720,0.658397,2058.72,70.990345,2020.56,69.674483,1.315862
7867,3119,2024,1571,1720,27,0,572,1563,137,480,...,0.285417,0.732323,0.452896,0.316049,0.736842,1842.24,68.231111,1806.56,66.909630,1.321481


In [44]:
#DReb_PCT
StatsTotals['DREB_PCT'] = StatsTotals['DR']/(StatsTotals['DR']+StatsTotals['Opp_OR'])
#OReb_PCT
StatsTotals['OREB_PCT'] = StatsTotals['OR']/(StatsTotals['OR']+StatsTotals['Opp_DR'])
#EFG_PCT
StatsTotals['EFG_PCT'] = (StatsTotals['FGM']+StatsTotals['FGM3']*0.5)/StatsTotals['FGA']
#Opp_EFG_PCT
StatsTotals['Opp_EFG_PCT'] = (StatsTotals['Opp_FGM']+StatsTotals['Opp_FGM3']*0.5)/StatsTotals['Opp_FGA']
#OFF_RTG
StatsTotals['OFF_RTG'] = 100*(StatsTotals['Score']/StatsTotals['POSS'])
#DEF_RTG
StatsTotals['DEF_RTG'] = 100*(StatsTotals['Opp_Score']/StatsTotals['Opp_POSS'])
#NET_RTG
StatsTotals['NET_RTG'] = StatsTotals['OFF_RTG']/StatsTotals['DEF_RTG']
#TOV_PCT
StatsTotals['TOV_PCT'] = StatsTotals['TO']/StatsTotals['POSS']

In [45]:
StatsTotals

,TeamID,Season,Score,Opp_Score,Games,NumOT,FGM,FGA,FGM3,FGA3,...,Opp_POSS_PER_GAME,POSS_Diff,DREB_PCT,OREB_PCT,EFG_PCT,Opp_EFG_PCT,OFF_RTG,DEF_RTG,NET_RTG,TOV_PCT
0,1101,2014,1326,1651,21,3,427,1053,140,375,...,66.546667,-0.080000,0.662016,0.249258,0.471985,0.564171,94.999284,118.141226,0.804116,0.225677
1,1101,2015,1708,2012,28,0,600,1482,203,537,...,64.280000,-0.195714,0.661853,0.241632,0.473347,0.553020,95.187142,111.787714,0.851499,0.200071
2,1101,2016,1886,2059,27,2,643,1456,185,509,...,69.502222,-0.788148,0.723810,0.245011,0.505151,0.545549,101.655815,109.722045,0.926485,0.195119
3,1101,2017,1697,1816,25,4,611,1333,177,477,...,68.312000,0.094400,0.682578,0.231902,0.524756,0.528689,99.230481,106.335636,0.933182,0.211676
4,1101,2018,1908,1966,27,1,689,1525,175,539,...,70.167407,-0.102222,0.700559,0.265217,0.509180,0.517394,100.858460,103.772987,0.971914,0.205629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,1657,1832,28,1,604,1563,173,567,...,67.221429,0.444286,0.719818,0.274266,0.441779,0.473716,87.457248,97.332908,0.898537,0.214817
12853,3476,2024,1566,2069,30,0,568,1662,165,597,...,69.824000,0.472000,0.642570,0.270729,0.391396,0.478425,74.257426,98.772151,0.751805,0.261750
12854,3477,2023,2068,2228,32,1,734,1863,117,456,...,74.122500,-0.680000,0.713352,0.325833,0.425389,0.482470,87.994009,93.932342,0.936781,0.227644
12855,3477,2024,1961,2151,29,1,709,1877,201,613,...,77.613793,-0.303448,0.675045,0.197851,0.431273,0.445368,87.466548,95.566021,0.915247,0.168153


In [46]:
StatsTotals.columns

Index(['TeamID', 'Season', 'Score', 'Opp_Score', 'Games', 'NumOT', 'FGM',
       'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
       'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM',
       'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl',
       'Opp_Blk', 'Opp_PF', 'FG_PCT', 'FG_PCT3', 'FT_PCT', 'Opp_FG_PCT',
       'Opp_FG_PCT3', 'Opp_FT_PCT', 'POSS', 'POSS_PER_GAME', 'Opp_POSS',
       'Opp_POSS_PER_GAME', 'POSS_Diff', 'DREB_PCT', 'OREB_PCT', 'EFG_PCT',
       'Opp_EFG_PCT', 'OFF_RTG', 'DEF_RTG', 'NET_RTG', 'TOV_PCT'],
      dtype='object')

## TO Do

Determine which columns to keep to be merged later into the season info dataset
Rename columns if necessary

In [47]:
ST = StatsTotals[['TeamID', 'Season', 'FG_PCT', 'FG_PCT3', 'FT_PCT', 'Opp_FG_PCT',
       'Opp_FG_PCT3', 'Opp_FT_PCT', 'POSS_PER_GAME',
       'Opp_POSS_PER_GAME', 'DREB_PCT', 'OREB_PCT', 'EFG_PCT',
       'Opp_EFG_PCT', 'OFF_RTG', 'DEF_RTG', 'NET_RTG', 'TOV_PCT']]

Games Data Doesnt need the stats anymore

In [48]:
RegSeasonGames.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF'],
      dtype='object')

In [49]:
RegSeasonGame = RegSeasonGames[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location']]
RegSeasonGame

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location
0,2003,10,1328,62,1104,68,N
1,2003,10,1272,70,1393,63,N
2,2003,11,1437,61,1266,73,N
3,2003,11,1457,50,1296,56,N
4,2003,11,1400,77,1208,71,N
...,...,...,...,...,...,...,...
189433,2024,131,3372,74,3465,75,N
189434,2024,132,3179,76,3283,75,N
189435,2024,132,3392,60,3180,68,N
189436,2024,132,3131,55,3221,61,H


Eventually ill Have to merge the RegSeasonGame with that teams season stats, I want to make some more advanced stats frist though
When I merge, Remember column names will ahve to be the same whcih will probably involve making 2 identical datasets out of the Stats Df, only differnce being the column names where 'TeamId' will become 'T1_TeamID' in one and 'T2_TeamID' in the other. All other columns will change similarly. For instance, 'Opp_FGM" becomes 'T1_Opp_FGM'


In [50]:
TeamRecords.shape

(12857, 11)

In [51]:
#Stats.shape

In [52]:
temp = pd.concat([WConf, MConf], ignore_index = True)
TeamRundown = pd.merge(TeamRecords, temp, 'left', on = ['Season', 'TeamID']).rename(columns = {'ConfAbbrev':'Conference'})
TeamRundown

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,Win_PCT,Conference
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,0.095238,southland
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,0.250000,southland
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,0.333333,southland
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,0.360000,southland
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,0.444444,southland
...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,0.000000,big_south
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,0.000000,acc
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,0.000000,big_sky
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,0.000000,horizon


#### Calculating Strength of schedule

Both for the full season and their conference tourney opponents

In [53]:
TeamRundown['Opp_wins'] = 0
TeamRundown['Opp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):
    for opp in TeamRundown['OpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
    for opp in TeamRundown['OpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
    for opp in TeamRundown['ConfOpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
    for opp in TeamRundown['ConfOpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'Opp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
    if row <= 4:
        print(TeamRundown['Opp_wins'][row])
        print(TeamRundown['Opp_losses'][row])
        

299
325
395
436
355
464
338
411
374
436


In [55]:
TeamRundown['Opp_win_pct'] = TeamRundown['Opp_wins']/(TeamRundown['Opp_wins']+TeamRundown['Opp_losses'])
TeamRundown    

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,Win_PCT,Conference,Opp_wins,Opp_losses,Opp_win_pct
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,0.095238,southland,299,325,0.479167
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,0.250000,southland,395,436,0.475331
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,0.333333,southland,355,464,0.433455
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,0.360000,southland,338,411,0.451268
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,0.444444,southland,374,436,0.461728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,0.000000,big_south,416,396,0.512315
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,0.000000,acc,59,49,0.546296
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,0.000000,big_sky,461,390,0.541716
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,0.000000,horizon,452,450,0.501109


In [56]:
TeamRundown['ConfOpp_wins'] = 0
TeamRundown['ConfOpp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):
    for opp in TeamRundown['ConfOpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'ConfOpp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'ConfOpp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
    for opp in TeamRundown['ConfOpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'ConfOpp_wins'] += TeamRundown.loc[mask, 'Wins'].sum() + TeamRundown.loc[mask, 'ConfWins'].sum()
        TeamRundown.at[row, 'ConfOpp_losses'] += TeamRundown.loc[mask, 'Losses'].sum() + TeamRundown.loc[mask, 'ConfLosses'].sum()
        


In [57]:
TeamRundown['ConfOpp_win_pct'] = TeamRundown['ConfOpp_wins']/(TeamRundown['ConfOpp_wins']+TeamRundown['ConfOpp_losses'])
TeamRundown  

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,Win_PCT,Conference,Opp_wins,Opp_losses,Opp_win_pct,ConfOpp_wins,ConfOpp_losses,ConfOpp_win_pct
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,0.095238,southland,299,325,0.479167,0,0,NaN
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,0.250000,southland,395,436,0.475331,0,0,NaN
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,0.333333,southland,355,464,0.433455,0,0,NaN
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,0.360000,southland,338,411,0.451268,0,0,NaN
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,0.444444,southland,374,436,0.461728,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,0.000000,big_south,416,396,0.512315,0,0,NaN
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,0.000000,acc,59,49,0.546296,0,0,NaN
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,0.000000,big_sky,461,390,0.541716,0,0,NaN
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,0.000000,horizon,452,450,0.501109,0,0,NaN


In [59]:

start_time = time.time()


TeamRundown['Opp_Opp_wins'] = 0
TeamRundown['Opp_Opp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):
    if row % 5000 == 0:
        print(row)
    for opp in TeamRundown['OpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    for opp in TeamRundown['OpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    for opp in TeamRundown['ConfOpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    for opp in TeamRundown['ConfOpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'Opp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'Opp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    if row <= 1:
        print(TeamRundown['Opp_Opp_wins'][row])
        print(TeamRundown['Opp_Opp_losses'][row])
        
        
        
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")

0
9707
9788
12052
13347
12008
13396
11003
11937
11720
13175
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
Execution time: 173.5309522151947 seconds


In [63]:
TeamRundown['Opp_Opp_win_pct'] = TeamRundown['Opp_Opp_wins']/(TeamRundown['Opp_Opp_wins']+TeamRundown['Opp_Opp_losses'])
TeamRundown   

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,...,Opp_losses,Opp_win_pct,ConfOpp_wins,ConfOpp_losses,ConfOpp_win_pct,Opp_Opp_wins,Opp_Opp_losses,Opp_Opp_win_pct,ConfOpp_Opp_wins,ConfOpp_Opp_losses
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,...,325,0.479167,0,0,NaN,9707,9788,0.497923,0,0
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,...,436,0.475331,0,0,NaN,12052,13347,0.474507,0,0
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,...,464,0.433455,0,0,NaN,12008,13396,0.472681,0,0
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,...,411,0.451268,0,0,NaN,11003,11937,0.479643,0,0
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,...,436,0.461728,0,0,NaN,11720,13175,0.470777,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,...,396,0.512315,0,0,NaN,11053,12826,0.462875,0,0
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,...,49,0.546296,0,0,NaN,1227,999,0.551213,0,0
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,...,390,0.541716,0,0,NaN,12234,12946,0.485862,0,0
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,...,450,0.501109,0,0,NaN,12958,13997,0.480727,0,0


In [64]:
TeamRundown['ConfOpp_Opp_wins'] = 0
TeamRundown['ConfOpp_Opp_losses'] = 0
count = 0
for row in range(len(TeamRundown)):   
    for opp in TeamRundown['ConfOpponentsLost'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'ConfOpp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'ConfOpp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()
    for opp in TeamRundown['ConfOpponentsBeaten'][row]:
        mask = (TeamRundown['Season'] == TeamRundown['Season'][row]) & (TeamRundown['TeamID'] == opp)
        TeamRundown.at[row, 'ConfOpp_Opp_wins'] += TeamRundown.loc[mask, 'Opp_wins'].sum()
        TeamRundown.at[row, 'ConfOpp_Opp_losses'] += TeamRundown.loc[mask, 'Opp_losses'].sum()

In [72]:
TeamRundown['ConfOpp_Opp_win_pct'] = TeamRundown['ConfOpp_Opp_wins']/(TeamRundown['ConfOpp_Opp_wins']+TeamRundown['ConfOpp_Opp_losses'])
TeamRundown   

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,...,ConfOpp_losses,ConfOpp_win_pct,Opp_Opp_wins,Opp_Opp_losses,Opp_Opp_win_pct,ConfOpp_Opp_wins,ConfOpp_Opp_losses,ConfOpp_Opp_win_pct,SOS,ConfSOS
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,...,0,NaN,9707,9788,0.497923,0,0,NaN,0.485419,NaN
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,...,0,NaN,12052,13347,0.474507,0,0,NaN,0.475056,NaN
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,...,0,NaN,12008,13396,0.472681,0,0,NaN,0.446531,NaN
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,...,0,NaN,11003,11937,0.479643,0,0,NaN,0.460726,NaN
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,...,0,NaN,11720,13175,0.470777,0,0,NaN,0.464745,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,...,0,NaN,11053,12826,0.462875,0,0,NaN,0.495835,NaN
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,...,0,NaN,1227,999,0.551213,0,0,NaN,0.547935,NaN
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,...,0,NaN,12234,12946,0.485862,0,0,NaN,0.523098,NaN
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,...,0,NaN,12958,13997,0.480727,0,0,NaN,0.494315,NaN


In [66]:
TeamRundown['SOS'] = (2*TeamRundown['Opp_win_pct']+TeamRundown['Opp_Opp_win_pct'])/3
TeamRundown['SOS'].describe()

count    12857.000000
mean         0.505756
std          0.042331
min          0.387594
25%          0.474839
50%          0.500276
75%          0.537118
max          0.637620
Name: SOS, dtype: float64

In [78]:
TeamRundown['ConfSOS'] = (2*TeamRundown['ConfOpp_win_pct']+TeamRundown['ConfOpp_Opp_win_pct'])/3
#Teams with no conf tourney games will probably play on  average similar opponents. I'm still going to bumb them down a bit though
for row in range(len(TeamRundown)):
    if pd.isnull(TeamRundown['ConfSOS'][row]):
        TeamRundown.loc[row, 'ConfSOS'] = TeamRundown['SOS'][row]*0.9
        
TeamRundown

C:\Users\18582\AppData\Local\Temp/ipykernel_11672/664357260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TeamRundown['ConfSOS'][row] = TeamRundown['SOS'][row]*0.9


,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,ConfWins,ConfOpponentsBeaten,ConfOpponentsLost,ConfLosses,...,ConfOpp_losses,ConfOpp_win_pct,Opp_Opp_wins,Opp_Opp_losses,Opp_Opp_win_pct,ConfOpp_Opp_wins,ConfOpp_Opp_losses,ConfOpp_Opp_win_pct,SOS,ConfSOS
0,1101,2014,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.0,[],[],0.0,...,0,NaN,9707,9788,0.497923,0,0,NaN,0.485419,0.436877
1,1101,2015,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.0,[],[],0.0,...,0,NaN,12052,13347,0.474507,0,0,NaN,0.475056,0.427551
2,1101,2016,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.0,[],[],0.0,...,0,NaN,12008,13396,0.472681,0,0,NaN,0.446531,0.401878
3,1101,2017,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.0,[],[],0.0,...,0,NaN,11003,11937,0.479643,0,0,NaN,0.460726,0.414654
4,1101,2018,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.0,[],[],0.0,...,0,NaN,11720,13175,0.470777,0,0,NaN,0.464745,0.418270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3421,2013,0.0,28.0,[],"[3441, 3459, 3184, 3314, 3378, 3144, 3367, 324...",0.0,[],[],0.0,...,0,NaN,11053,12826,0.462875,0,0,NaN,0.495835,0.446252
12853,3438,2021,0.0,5.0,[],"[3416, 3187, 3241, 3155, 3199]",0.0,[],[],0.0,...,0,NaN,1227,999,0.551213,0,0,NaN,0.547935,0.493142
12854,3451,2013,0.0,29.0,[],"[3140, 3129, 3329, 3119, 3428, 3307, 3430, 346...",0.0,[],[],0.0,...,0,NaN,12234,12946,0.485862,0,0,NaN,0.523098,0.470788
12855,3464,2010,0.0,30.0,[],"[3189, 3219, 3245, 3145, 3419, 3451, 3338, 310...",0.0,[],[],0.0,...,0,NaN,12958,13997,0.480727,0,0,NaN,0.494315,0.444883


In [79]:
TeamRundown.columns

Index(['TeamID', 'Season', 'Wins', 'Losses', 'OpponentsBeaten',
       'OpponentsLost', 'ConfWins', 'ConfOpponentsBeaten', 'ConfOpponentsLost',
       'ConfLosses', 'Win_PCT', 'Conference', 'Opp_wins', 'Opp_losses',
       'Opp_win_pct', 'ConfOpp_wins', 'ConfOpp_losses', 'ConfOpp_win_pct',
       'Opp_Opp_wins', 'Opp_Opp_losses', 'Opp_Opp_win_pct', 'ConfOpp_Opp_wins',
       'ConfOpp_Opp_losses', 'ConfOpp_Opp_win_pct', 'SOS', 'ConfSOS'],
      dtype='object')

In [80]:
TR = TeamRundown[['TeamID', 'Season', 'Wins', 'Losses', 'Conference','SOS', 'ConfWins', 'ConfLosses', 'ConfSOS']]

In [81]:
print(TR.shape, SA.shape, ST.shape)

(12857, 9) (12857, 22) (12857, 18)


### Time to merge Our 3 dfs containg team season stats

In [82]:
temp1 = pd.merge(TR, SA, 'left', on = ['TeamID', 'Season'])

In [83]:
Teams = pd.merge(temp1, ST, 'left', on = ['TeamID', 'Season'])

In [84]:
Teams.shape


(12857, 45)

In [85]:
Teams.to_csv('Data\\TeamInfo.csv', index=False)